In [1]:
import os

os.environ["WF_TRACE_SERVER_URL"] = "http://127.0.01:6345"

In [2]:
import weave

In [3]:
client = weave.init("remote_model_demo_4")

Logged in as Weights & Biases user: timssweeney.
View Weave data at https://wandb.ai/timssweeney/remote_model_demo_4/weave


In [4]:
# Demonstrates creating a model in python

from weave.builtin_objects.models.CompletionModel import LiteLLMCompletionModel

model = LiteLLMCompletionModel(
    model="gpt-4o",
    messages_template=[
        {
            "role": "system",
            "content": "Please extract the name and age from the following text",
        },
        {"role": "user", "content": "{user_input}"},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Person",
            "schema": {
                "type": "object",
                "properties": {
                    "age": {"type": "integer"},
                    "name": {"type": "string"},
                },
            },
        },
    },
)

res = model.predict(user_input="Hello, my name is Fred and I am 30 years old.")

print(model.ref.uri())

print(res)

🍩 https://wandb.ai/timssweeney/remote_model_demo_4/r/call/0193ba4f-de91-79a2-9028-67c3c500703e
weave:///timssweeney/remote_model_demo_4/object/LiteLLMCompletionModel:KBsfUswVpEHFYmZuJjmhM2YH4EttkRZJSoH0Z0ZaNRY
{'name': 'Fred', 'age': 30}


In [5]:
# Demonstrates calling a model created in python

from weave.trace_server.trace_server_interface import CallMethodReq

call_res = client.server.call_method(
    CallMethodReq(
        project_id=client._project_id(),
        object_ref=model.ref.uri(),
        method_name="predict",
        args={"user_input": "Hello, my name is Charles and I am 40 years old."},
    )
)

call_res

CallMethodRes(call_id='0193ba4f-fc13-79c2-b217-03e6fdd7e7c4', output={'name': 'Charles', 'age': 40})

In [6]:
# Demonstrates creating a model in the UI - notice the digest match

from weave.trace_server.trace_server_interface import ObjCreateReq

obj_res = client.server.obj_create(
    ObjCreateReq.model_validate(
        {
            "obj": {
                "project_id": client._project_id(),
                "object_id": "LiteLLMCompletionModel",
                "val": {
                    "model": "gpt-4o",
                    "messages_template": [
                        {
                            "role": "system",
                            "content": "Please extract the name and age from the following text!",
                        },
                        {"role": "user", "content": "{user_input}"},
                    ],
                    "response_format": {
                        "type": "json_schema",
                        "json_schema": {
                            "name": "Person",
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "age": {"type": "integer"},
                                    "name": {"type": "string"},
                                },
                            },
                        },
                    },
                },
                "set_leaf_object_class": "LiteLLMCompletionModel",
            }
        }
    )
)

obj_res

ObjCreateRes(digest='k85wXnWLVxpHujpohAqNBIirXZSM6XRSOSk84n1XR84')

In [7]:
# Demonstrates fetching a model in python that was created in the UI

from weave.trace.refs import ObjectRef

gotten_model = weave.ref(
    ObjectRef(
        entity=client.entity,
        project=client.project,
        name="LiteLLMCompletionModel",
        _digest=obj_res.digest,
    ).uri()
).get()

res = gotten_model.predict(user_input="Hello, my name is Fred and I am 30 years old.")
res

🍩 https://wandb.ai/timssweeney/remote_model_demo_4/r/call/0193ba50-1be0-70d2-84cd-dcf8fac3ff09


{'name': 'Fred', 'age': 30}

In [8]:
gotten_model

LiteLLMCompletionModel(name=None, description=None, model='gpt-4o', messages_template=WeaveList([{'role': 'system', 'content': 'Please extract the name and age from the following text!'}, {'role': 'user', 'content': '{user_input}'}]), response_format=WeaveDict({'type': 'json_schema', 'json_schema': {'name': 'Person', 'schema': {'type': 'object', 'properties': {'age': {'type': 'integer'}, 'name': {'type': 'string'}}}}}))

In [9]:
# Part 2: Scoring:

In [12]:
import weave
from weave.trace.refs import CallRef
from weave.trace_server import trace_server_interface as tsi

obj_create_res = client.server.obj_create(
    tsi.ObjCreateReq.model_validate(
        {
            "obj": {
                "project_id": client._project_id(),
                "object_id": "CorrectnessJudge",
                "val": {
                    "model": "gpt-4o",
                    "system_prompt": "You are a judge that scores the correctness of a response.",
                    "response_format": {
                        "type": "json_schema",
                        "json_schema": {
                            "name": "Correctness",
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "is_correct": {"type": "boolean"},
                                },
                            },
                        },
                    },
                },
                "set_leaf_object_class": "LLMJudgeScorer",
            }
        }
    )
)
client._flush()
scorer_ref = weave.ObjectRef(
    entity=client._project_id().split("/")[0],
    project=client._project_id().split("/")[1],
    name="CorrectnessJudge",
    _digest=obj_create_res.digest,
)

call_ref = CallRef(
    entity=client._project_id().split("/")[0],
    project=client._project_id().split("/")[1],
    id=call_res.call_id,
)

score_res = client.server.score_call(
    tsi.ScoreCallReq.model_validate(
        {
            "project_id": client._project_id(),
            "call_ref": call_ref.uri(),
            "scorer_ref": scorer_ref.uri(),
        }
    )
)

score_res

ScoreCallRes(feedback_id='0193ba55-d466-74a3-a4de-da0a456b08a7', score_call=CallSchema(id='0193ba55-cb43-7c61-a712-f7249e6dfe4f', project_id='UHJvamVjdEludGVybmFsSWQ6NDA1NzYyOTQ=', op_name='weave:///timssweeney/remote_model_demo_4/op/LLMJudgeScorer.score:LSxb3VBdL8YmPr9vqYhxsMe74D8C04dJL1IKQ61Ke7M', display_name=None, trace_id='0193ba55-cb43-7c61-a712-f71512a66d3b', parent_id=None, started_at=datetime.datetime(2024, 12, 12, 10, 6, 45, 59887, tzinfo=TzInfo(UTC)), attributes={'weave': {'client_version': '0.51.25-dev0', 'source': 'python-sdk', 'os_name': 'Darwin', 'os_version': 'Darwin Kernel Version 23.2.0: Wed Nov 15 21:53:18 PST 2023; root:xnu-10002.61.3~2/RELEASE_ARM64_T6000', 'os_release': '23.2.0', 'sys_version': '3.10.8 (main, Dec  5 2022, 18:10:41) [Clang 14.0.0 (clang-1400.0.29.202)]'}}, inputs={'self': 'weave:///timssweeney/remote_model_demo_4/object/LLMJudgeScorer:uCL086uULzE1HKLFn8YIezCG98HiqayaAp3d1R9ktA0', 'inputs': {'kwargs': {'user_input': 'Hello, my name is Charles and I 